In [2]:
# Cài đặt thư viện
%pip install -q ultralytics

from ultralytics import YOLO
import torch
import matplotlib.pyplot as plt
import yaml
import os
from pathlib import Path

# Kiểm tra thiết bị
print(f"Using CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name()}")
print(f"PyTorch version: {torch.__version__}")

Note: you may need to restart the kernel to use updated packages.
Using CUDA: False
PyTorch version: 2.6.0+cpu


In [ ]:
# Đường dẫn tới file cấu hình dataset
dataset_yaml = '../datasets/license_plate_ocr/dataset.yaml'

# Kiểm tra và hiển thị cấu hình dataset
with open(dataset_yaml, 'r') as f:
    cfg = yaml.safe_load(f)
print("Dataset configuration:")
print(f"Training images: {cfg['train']}")
print(f"Validation images: {cfg['val']}")
print(f"Number of classes: {cfg['nc']}")
print("Classes:", cfg['names'])

In [ ]:
# Khởi tạo model từ YOLOv8n
model = YOLO('yolov8n.pt')

# Training với cấu hình tối ưu cho OCR
results = model.train(
    data=dataset_yaml,
    epochs=100,
    imgsz=640,
    batch=32 if torch.cuda.is_available() else 4,
    device='0' if torch.cuda.is_available() else 'cpu',
    name='LP_ocr_v8',
    cache=True,
    # Augmentation tối ưu cho OCR
    hsv_h=0.015,  # Ít thay đổi màu sắc
    hsv_s=0.3,
    hsv_v=0.2,
    degrees=2.0,   # Xoay nhẹ
    translate=0.1,
    scale=0.2,
    shear=0.0,     # Không shear
    perspective=0.0, # Không perspective
    flipud=0.01,    # Hạn chế lật
    fliplr=0.01,
    mosaic=0.5,     # Giảm mosaic
    mixup=0.0,      # Tắt mixup
    copy_paste=0.0,  # Tắt copy-paste
    # Training parameters
    warmup_epochs=3,
    patience=20,    # Early stopping
    lr0=0.01,      # Initial learning rate
    lrf=0.001,     # Final learning rate
    momentum=0.937,
    weight_decay=0.0005,
    # Loss weights
    box=7.5,       # Box loss gain
    cls=0.3,       # Class loss gain
    dfl=1.5,       # DFL loss gain
    close_mosaic=10,# Tắt mosaic trong 10 epoch cuối
    # Validation
    val=True,      # Validate sau mỗi epoch
    # Save checkpoints
    save_period=10, # Lưu mỗi 10 epochs
    exist_ok=True,  # Ghi đè thư mục cũ
)

In [ ]:
# Vẽ kết quả training
fig, ax = plt.subplots(2, 3, figsize=(20, 10))

# Plot losses
ax[0,0].plot(results.results_dict['train/box_loss'], label='train')
ax[0,0].plot(results.results_dict['val/box_loss'], label='val')
ax[0,0].set_title('Box Loss')
ax[0,0].set_xlabel('Epoch')
ax[0,0].legend()

ax[0,1].plot(results.results_dict['train/cls_loss'], label='train')
ax[0,1].plot(results.results_dict['val/cls_loss'], label='val')
ax[0,1].set_title('Classification Loss')
ax[0,1].set_xlabel('Epoch')
ax[0,1].legend()

ax[0,2].plot(results.results_dict['train/dfl_loss'], label='train')
ax[0,2].plot(results.results_dict['val/dfl_loss'], label='val')
ax[0,2].set_title('DFL Loss')
ax[0,2].set_xlabel('Epoch')
ax[0,2].legend()

# Plot metrics
ax[1,0].plot(results.results_dict['metrics/precision'], label='precision')
ax[1,0].plot(results.results_dict['metrics/recall'], label='recall')
ax[1,0].set_title('Precision & Recall')
ax[1,0].set_xlabel('Epoch')
ax[1,0].legend()

ax[1,1].plot(results.results_dict['metrics/mAP50(B)'], label='mAP50')
ax[1,1].plot(results.results_dict['metrics/mAP50-95(B)'], label='mAP50-95')
ax[1,1].set_title('Mean Average Precision')
ax[1,1].set_xlabel('Epoch')
ax[1,1].legend()

if 'metrics/fitness' in results.results_dict:
    ax[1,2].plot(results.results_dict['metrics/fitness'])
    ax[1,2].set_title('Model Fitness')
    ax[1,2].set_xlabel('Epoch')

plt.tight_layout()
plt.show()

In [ ]:
# Kiểm tra kết quả trên một số ảnh biển số
best_model_path = 'runs/detect/LP_OCR_yolov8_new/weights/best.pt'
model = YOLO(best_model_path)

# Sử dụng ảnh test từ thư mục test_images
test_images = ['../test_image/bien_so.jpg']

for img_path in test_images:
    results = model.predict(img_path, conf=0.25)
    
    # Hiển thị kết quả
    plt.figure(figsize=(10, 4))
    plt.imshow(results[0].plot())
    plt.axis('off')
    plt.show()